In [20]:
import pandas as pd
import urllib.request

#Librerias para conexión con BD.
import pymysql
from sqlalchemy import create_engine
import psycopg2

1.

In [13]:
url = "https://cdn.buenosaires.gob.ar/datosabiertos/datasets/ente-de-turismo/oferta-establecimientos-gastronomicos/oferta_gastronomica.csv"

In [14]:
urllib.request.urlretrieve(url, "oferta_gastronomica_BA.csv")

('oferta_gastronomica_BA.csv', <http.client.HTTPMessage at 0x1fbd5d9ac00>)

In [18]:
dataset = pd.read_csv(url)

In [19]:
dataset.columns

Index(['long', 'lat', 'id', 'nombre', 'categoria', 'cocina', 'ambientacion',
       'telefono', 'mail', 'horario', 'calle_nombre', 'calle_altura',
       'calle_cruce', 'direccion_completa', 'barrio', 'comuna',
       'codigo_postal', 'codigo_postal_argentino'],
      dtype='object')

In [85]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2823 entries, 0 to 2822
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   long                     2823 non-null   float64
 1   lat                      2823 non-null   float64
 2   id                       2823 non-null   int64  
 3   nombre                   2823 non-null   object 
 4   categoria                2707 non-null   object 
 5   cocina                   2464 non-null   object 
 6   ambientacion             2264 non-null   object 
 7   telefono                 2303 non-null   object 
 8   mail                     472 non-null    object 
 9   horario                  725 non-null    object 
 10  calle_nombre             2823 non-null   object 
 11  calle_altura             2823 non-null   int64  
 12  calle_cruce              6 non-null      object 
 13  direccion_completa       2822 non-null   object 
 14  barrio                  

2.

In [158]:
conexion = pymysql.connect(
    host = 'localhost',
    database = 'tarea_8',
    user = 'root',
    password = 'root123'
)

cursor = conexion.cursor()

In [146]:
comandos = [
    'USE tarea_8;',
    'CREATE TABLE datos (id_local INT NOT NULL AUTO_INCREMENT PRIMARY KEY, nombre VARCHAR(50), categoria VARCHAR(50), direccion VARCHAR(50), barrio VARCHAR(50), comuna VARCHAR(30));',
    'SELECT * FROM datos;'
]

In [147]:
for i in comandos:
    cursor.execute(i)

In [117]:
cursor.execute('ALTER TABLE datos CHANGE nombre nombre VARCHAR(50);')
cursor.execute('ALTER TABLE datos CHANGE direccion direccion VARCHAR(50);')
cursor.execute('ALTER TABLE datos CHANGE barrio barrio VARCHAR(50);')
cursor.execute('ALTER TABLE datos CHANGE comuna comuna VARCHAR(50);')

0

In [149]:
nombre = list(dataset['nombre'])
categoria = list(dataset['categoria'])
direccion = list(dataset['direccion_completa'])
barrio = list(dataset['barrio'])
comuna = list(dataset['comuna'])

print('Longitud de la lista nombre:', len(nombre))
print('Longitud de la lista categoria:', len(categoria))
print('Longitud de la lista dirección:', len(direccion))
print('Longitud de la lista barrio', len(barrio))
print('Longitud de la lista comuna', len(comuna))

Longitud de la lista nombre: 2823
Longitud de la lista categoria: 2823
Longitud de la lista dirección: 2823
Longitud de la lista barrio 2823
Longitud de la lista comuna 2823


In [97]:
str(direccion[2822])

'nan'

In [150]:
for i in range(len(direccion)):
    if type(direccion[i]) != str:
        direccion[i] = str(direccion[i])
        #print('Indice:', i, 'Tipo', type(direccion[i]))

    direccion[i] = direccion[i].replace(',', '')
    direccion[i] = direccion[i].replace("'", ' ')

In [151]:
for i in range(len(nombre)):
    nombre[i] = nombre[i].replace("'", ' ')

In [152]:
comandos2 = []

for i in range(len(nombre)):
    cadena = f"INSERT datos (nombre, categoria, direccion, barrio, comuna) VALUES ('{nombre[i]}', '{categoria[i]}', '{direccion[i]}', '{barrio[i]}', '{comuna[i]}');"
    comandos2.append(cadena)

In [154]:
len(comandos2)

2823

In [159]:
for i in comandos2:
    cursor.execute(i)

conexion.commit()
conexion.close()

In [119]:
for i in comandos2:
    if "'BRIEN 1170', 'Constitucion', 'Comuna 1'" in i:
        print('Indice:', i)

Indice: INSERT datos (nombre, direccion, barrio, comuna) VALUES ('LA CUBANA', 'O'BRIEN 1170', 'Constitucion', 'Comuna 1');


3.
a) ¿Cuál es el barrio con mayor cantidad de Pubs?

b) Obtener la cantidad de locales por categoría

c) Obtener la cantidad de restaurantes por comuna

a

In [160]:
conexion = pymysql.connect(
    host = 'localhost',
    database = 'tarea_8',
    user = 'root',
    password = 'root123'
)

cursor = conexion.cursor()

In [168]:
consulta = "SELECT barrio, COUNT(*) AS cantidad FROM datos WHERE categoria = 'pub' GROUP BY barrio ORDER BY cantidad DESC LIMIT 1"

print(cursor.execute(consulta))

1


In [169]:
for i in cursor:
    print(i)

('Recoleta', 8)


b

In [170]:
consulta = "SELECT categoria, COUNT(*) AS cantidad FROM datos GROUP BY categoria ORDER BY cantidad DESC"

print(cursor.execute(consulta))

9


In [171]:
for i in cursor:
    print(i)

('RESTAURANTE', 1990)
('CAFE', 334)
('BAR', 247)
('nan', 116)
('CONFITERIA', 55)
('PUB', 53)
('VINERIA', 22)
('SANDWICHERIA', 4)
('DELIVERY & TAKE AWAY', 2)


c

In [172]:
consulta = "SELECT comuna, COUNT(*) AS cantidad FROM datos WHERE categoria = 'restaurante' GROUP BY comuna ORDER BY cantidad DESC"

print(cursor.execute(consulta))

15


In [173]:
for i in cursor:
    print(i)

('Comuna 1', 609)
('Comuna 14', 366)
('Comuna 2', 227)
('Comuna 3', 145)
('Comuna 13', 108)
('Comuna 4', 95)
('Comuna 6', 87)
('Comuna 7', 64)
('Comuna 15', 57)
('Comuna 5', 56)
('Comuna 9', 52)
('Comuna 10', 43)
('Comuna 12', 42)
('Comuna 11', 33)
('Comuna 8', 6)
